In [116]:
import os
import json
import openai
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI
load_dotenv(override=True)

aoai_client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-07-01-preview"
)

In [198]:
names_df = pd.read_csv("../data/raw/US_name_list.csv",names=["First","Last","Gender","extra"]).drop(columns = ['Gender','extra']).head(200)

In [212]:
#https://github.com/derek73/python-nameparser
instructions = """ You are a name generation agent, please generate random names.

The supported name structures are the following:

Comma-separated format like "Last, First" is also supported.
Firstname Middle Lastname Suffix
Lastname [Suffix], Title Firstname Middle
Lastname Last Name, Title Firstname Middle
Lastname , Title Firstname Middle
Firstname Middlename Lastname
Title Firstname Middlename Lastname


##Important
Please provide the output in a json with the key full_name"""

def GenerateFullName(full_name):
    query = "Please generate a full name blob by using the following dictionary containing first and last time" + full_name
    messages = [{"role":"system","content":instructions}, 
               {"role":"user","content":query}]

    response = aoai_client.chat.completions.create(model="gpt4o",  
                                        messages = messages, 
                                        temperature=0.5,  
                                        max_tokens=2000,
                                        response_format={ "type": "json_object" })
                                        #seed = 42)
    period = response.choices[0].message.content

    return json.loads(period)['full_name']

In [213]:
names_df['full_name'] = names_df.apply(lambda x: GenerateFullName(x.to_json()), axis=1)

In [219]:
names_df.head(180)

,First,Last,full_name
0,Chelsea,Mitchell,"Mitchell, Dr. Chelsea R., Jr., PhD"
1,Brandon,Sylvester,"Sylvester, Dr. Brandon T. J. Avery, PhD, Esq."
2,Chris,Toussaint,"Toussaint, Dr. Chris A. M. ""Kit"", Jr."
3,Willie,Gotti,"Gotti, Willie"
4,Cristobal,Corona,"Corona, Dr. Cristobal J. III"
...,...,...,...
175,Aml S,As,"As, Dr. Aml S 'Ace' Jordan Michael, Jr."
176,Trey,Condon,"Condon, Trey M"
177,Jihad,Shammari,"Shammari, Dr. Jihad Alexei Mahmoud, II"
178,Muhammad,Ayub,"Ayub, Dr. Muhammad Ali"


In [215]:
names_df.to_parquet("../data/processed/full_names.parquet", index=False)

In [216]:
for i in range(len(names_df)):
    path = f"../data/processed/text/full_names_{i}.txt"
    with open(path, "w") as f:
        try:
            f.write(names_df.iloc[i]["full_name"])
        except:
            print(f"Error writing to {path}")

Error writing to ../data/processed/text/full_names_37.txt
Error writing to ../data/processed/text/full_names_40.txt
Error writing to ../data/processed/text/full_names_41.txt
Error writing to ../data/processed/text/full_names_48.txt
Error writing to ../data/processed/text/full_names_54.txt
Error writing to ../data/processed/text/full_names_64.txt
Error writing to ../data/processed/text/full_names_68.txt
Error writing to ../data/processed/text/full_names_88.txt
Error writing to ../data/processed/text/full_names_89.txt
Error writing to ../data/processed/text/full_names_90.txt
Error writing to ../data/processed/text/full_names_93.txt
Error writing to ../data/processed/text/full_names_111.txt
Error writing to ../data/processed/text/full_names_119.txt
Error writing to ../data/processed/text/full_names_122.txt
Error writing to ../data/processed/text/full_names_130.txt
Error writing to ../data/processed/text/full_names_131.txt
Error writing to ../data/processed/text/full_names_139.txt
Error wr